In [ ]:
pip install transformers

     |████████████████████████████████| 2.5MB 9.7MB/s 
     |████████████████████████████████| 901kB 46.2MB/s 
     |████████████████████████████████| 3.3MB 47.5MB/s 


In [ ]:
pip install -U deep_translator

In [ ]:
# import library
import json
import numpy as np
import torch
import os
import textwrap
import time
import datetime
from transformers import BertTokenizer, BertForQuestionAnswering, AdamW, BertConfig, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, random_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open(os.path.join('/content/drive/MyDrive/train-v1.1.json'), "r", encoding="utf-8") as reader:
    input_data = json.load(reader)["data"]
dataset = []
for sample in input_data:
    title = sample['title']
    paragraphs = sample['paragraphs']
    for paragraph in paragraphs:
        context = GoogleTranslator(source='auto', target='fa').translate(paragraph['context'])
        qas = paragraph['qas']
        for qa in qas:
            sample = {}
            sample["title"] = title
            sample["context"] = context
            sample["question"] = GoogleTranslator(source='en', target='fa').translate(qa['question'])

            sample["id"] = qa['id']
            answer = qa['answers'][0]
            if answer['text'].isdigit():
                sample["answer_text"] = answer['text']
            else:
                sample["answer_text"] = GoogleTranslator(source='auto', target='fa').translate(answer['text'])

            sample["answer_start"] = answer['answer_start']
            dataset.append(sample)

In [ ]:
wrapper = textwrap.TextWrapper(width=80) 

ex = dataset[888]

print('Title:', ex['title'])
print('ID:', ex['id'])

print('\n======== Question =========')
print(ex['question'])

print('\n======== Context =========')
print(wrapper.fill(ex['context']))

print('\n======== Answer =========')
print(ex['answer_text'])

Title: Beyoncé
ID: 56becb8d3aeaaa14008c9499

======== Question =========
چه کسی اولین زن موفق به دریافت جایزه بین المللی هنرمند در جوایز موسیقی آمریکا بود؟

======== Context =========
بیانسه جوایز بی شماری دریافت کرده است. وی به عنوان یک هنرمند انفرادی بیش از 15
میلیون آلبوم در ایالات متحده و بیش از 118 میلیون آلبوم در سراسر جهان فروخته است
(60 میلیون مورد دیگر با Destiny's Child) ، او را به یکی از پرفروش ترین هنرمندان
موسیقی در تمام دوران تبدیل کرده است. انجمن صنعت ضبط امریکا (RIAA) بیانسه را به
عنوان برترین هنرمند دارای گواهینامه 2000s با 64 گواهینامه معرفی کرد. آهنگ های
"Crazy in Love" ، "Single Ladies (Put a ring on it))" ، "Halo" و "Irreplaceable"
از پرفروش ترین تک آهنگ های تمام ادوار جهان است. در سال 2009 ، The Observer او را
به عنوان Artist of the Decade و Billboard او را به عنوان هنرمند برتر زن و برترین
آهنگ های رادیویی دهه معرفی کرد. در سال 2010 ، بیلبورد او را در لیست "50 هنرمند
برتر 25 سال گذشته R & B / Hip-Hop" با شماره 15 قرار داد. در سال 2012 VH1 در رتبه
سوم خود در لیست "

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained(
    'bert-base-cased',
    do_lower_case=False
)

In [ ]:
for elem in dataset[0]:
  print(elem)

title
context
question
id
answer_text
answer_start


In [ ]:
def format_time(elapsed):

    elapsed_rounded = int(round((elapsed)))
    
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
t0 = time.time()

all_input_ids = []
attention_masks = []
segment_ids = [] 
start_positions = []
end_positions = []

num_dropped = 0

update_interval = 10000

print('Tokenizing {:,} examples...'.format(len(dataset)))

for (ex_num, ex) in enumerate(dataset):

    if (ex_num % update_interval) == 0 and not (ex_num == 0):
        
        elapsed = format_time(time.time() - t0)
        
        ex_per_sec = (time.time() - t0) / ex_num
        remaining_sec = ex_per_sec * (len(dataset) - ex_num)
        remaining = format_time(remaining_sec)

        print('  Example {:>7,}  of  {:>7,}.    Elapsed: {:}. Remaining: {:}'.format(ex_num, len(dataset), elapsed, remaining))


    answer_tokens = tokenizer.tokenize(ex['answer_text'])

    sentinel_str = ' '.join(['[MASK]']*len(answer_tokens))

    start_char_i = ex['answer_start']
    end_char_i = start_char_i + len(ex['answer_text'])
    context_w_sentinel = ex['context'][:start_char_i] + \
                         sentinel_str + \
                         ex['context'][end_char_i:]

    encoded_dict = tokenizer.encode_plus(
        ex['question'], 
        context_w_sentinel,
        add_special_tokens = True,
        max_length = 384,
        pad_to_max_length = True,
        truncation = True,
        return_attention_mask = True,
        return_tensors = 'pt', 
    )

    input_ids = encoded_dict['input_ids']

    is_mask_token = (input_ids[0] == tokenizer.mask_token_id)

    mask_token_indeces = is_mask_token.nonzero(as_tuple=False)[:, 0]

    if not len(mask_token_indeces) == len(answer_tokens):
        
        num_dropped += 1

        continue


    start_index = mask_token_indeces[0]
    end_index = mask_token_indeces[-1]

    answer_token_ids = tokenizer.encode(answer_tokens, 
                                        add_special_tokens=False, 
                                        return_tensors='pt')

    input_ids[0, start_index : end_index + 1] = answer_token_ids
   
    all_input_ids.append(input_ids)

    attention_masks.append(encoded_dict['attention_mask'])    

    segment_ids.append(encoded_dict['token_type_ids'])

    start_positions.append(start_index)
    end_positions.append(end_index)

all_input_ids = torch.cat(all_input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
segment_ids = torch.cat(segment_ids, dim=0)

start_positions = torch.tensor(start_positions)
end_positions = torch.tensor(end_positions)

print('DONE.  Tokenization took {:}'.format(format_time(time.time() - t0)))

Tokenizing 87,599 examples...


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2132: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


  Example  10,000  of   87,599.    Elapsed: 0:00:27. Remaining: 0:03:27
  Example  20,000  of   87,599.    Elapsed: 0:00:49. Remaining: 0:02:45
  Example  30,000  of   87,599.    Elapsed: 0:01:14. Remaining: 0:02:23
  Example  40,000  of   87,599.    Elapsed: 0:01:44. Remaining: 0:02:04
  Example  50,000  of   87,599.    Elapsed: 0:02:12. Remaining: 0:01:39
  Example  60,000  of   87,599.    Elapsed: 0:02:41. Remaining: 0:01:14
  Example  70,000  of   87,599.    Elapsed: 0:03:09. Remaining: 0:00:48
  Example  80,000  of   87,599.    Elapsed: 0:03:38. Remaining: 0:00:21
DONE.  Tokenization took 0:04:01


In [ ]:
print('Tokenized {:,} examples.'.format(len(all_input_ids)))

print('\nDropped {:,} examples.'.format(num_dropped))

Tokenized 87,502 examples.

Dropped 97 examples.


In [ ]:
dataset = TensorDataset(all_input_ids, 
                        attention_masks, 
                        segment_ids, 
                        start_positions, 
                        end_positions)

print('Dataset size: {:} samples'.format(len(dataset)))

Dataset size: 87502 samples


In [ ]:
train_size = int(0.98 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

85,751 training samples
1,751 validation samples


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SubsetRandomSampler, SequentialSampler

import numpy.random
import numpy as np

batch_size = 16

train_dataloader = DataLoader(
            train_dataset, 
            sampler = RandomSampler(train_dataset),
            batch_size = batch_size
        )

validation_dataloader = DataLoader(
            val_dataset,
            sampler = SequentialSampler(val_dataset),
            batch_size = batch_size 
        )

print('{:,} training batches & {:,} validation batches'.format(len(train_dataloader), len(validation_dataloader)))

5,360 training batches & 110 validation batches


In [ ]:
model = BertForQuestionAnswering.from_pretrained(
    "bert-base-cased", 
    output_attentions = False, 
    output_hidden_states = False
)

desc = model.cuda()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and a

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8
                )

epochs = 3

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)


In [ ]:
import random
import numpy as np
device = torch.device("cuda")

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

training_stats = []


for epoch_i in range(0, epochs):
    

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    
    print('Training {:,} batches...'.format(len(train_dataloader)))

    t0 = time.time()

    total_train_loss = 0

    model.train()

    update_interval = 100


    num_batches = len(train_dataloader)

    for step, batch in enumerate(train_dataloader):

        if step % update_interval == 0 and not step == 0:
            
            elapsed = format_time(time.time() - t0)
            
            step_per_sec = (time.time() - t0) / step
            remaining_sec = step_per_sec * (num_batches - step)
            remaining = format_time(remaining_sec)

            print('  Batch {:>7,}  of  {:>7,}.    Elapsed: {:}. Remaining: {:}'.format(step, num_batches, elapsed, remaining))

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_seg_ids = batch[2].to(device)
        b_start_pos = batch[3].to(device)
        b_end_pos = batch[4].to(device)

        model.zero_grad()        

        outputs = model(b_input_ids, 
                        attention_mask=b_input_mask, 
                        token_type_ids = b_seg_ids,
                        start_positions=b_start_pos,
                        end_positions=b_end_pos)

        loss = outputs[0]
        start_logits = outputs[1]
        end_logits = outputs[2]

        total_train_loss += loss.item()

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()

        scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
       

    print("")
    print("Running Validation...")

    model.eval()

    total_eval_accuracy = 0
    total_eval_loss = 0

    t0_val = time.time()

    pred_start, pred_end, true_start, true_end = [], [], [], []

    for batch in validation_dataloader:

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_seg_ids = batch[2].to(device)
        b_start_pos = batch[3].to(device)
        b_end_pos = batch[4].to(device)
       
        with torch.no_grad():        

            outputs = model(b_input_ids, 
                            token_type_ids=b_seg_ids, 
                            attention_mask=b_input_mask,
                            start_positions=b_start_pos,
                            end_positions=b_end_pos)

        loss = outputs[0]
        start_logits = outputs[1]
        end_logits = outputs[2]
        total_eval_loss += loss.item()

        start_logits = start_logits.detach().cpu().numpy()
        end_logits = end_logits.detach().cpu().numpy()
        
        b_start_pos = b_start_pos.to('cpu').numpy()
        b_end_pos = b_end_pos.to('cpu').numpy()

        answer_start = np.argmax(start_logits, axis=1)
        answer_end = np.argmax(end_logits, axis=1)

        pred_start.append(answer_start)
        pred_end.append(answer_end)
        true_start.append(b_start_pos)
        true_end.append(b_end_pos)

    pred_start = np.concatenate(pred_start, axis=0)
    pred_end = np.concatenate(pred_end, axis=0)
    true_start = np.concatenate(true_start, axis=0)
    true_end = np.concatenate(true_end, axis=0)
        
    num_start_correct = np.sum(pred_start == true_start)
    num_end_correct = np.sum(pred_end == true_end)

    total_correct = num_start_correct + num_end_correct
    total_indeces = len(true_start) + len(true_end)

    avg_val_accuracy = float(total_correct) / float(total_indeces)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    validation_time = format_time(time.time() - t0_val)
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")


======== Epoch 1 / 3 ========
Training 5,360 batches...
  Batch     100  of    5,360.    Elapsed: 0:02:04. Remaining: 1:48:53
  Batch     200  of    5,360.    Elapsed: 0:04:21. Remaining: 1:52:03
  Batch     300  of    5,360.    Elapsed: 0:06:37. Remaining: 1:51:35
  Batch     400  of    5,360.    Elapsed: 0:08:53. Remaining: 1:50:13
  Batch     500  of    5,360.    Elapsed: 0:11:09. Remaining: 1:48:27
  Batch     600  of    5,360.    Elapsed: 0:13:26. Remaining: 1:46:34
  Batch     700  of    5,360.    Elapsed: 0:15:42. Remaining: 1:44:30
  Batch     800  of    5,360.    Elapsed: 0:17:58. Remaining: 1:42:27
  Batch     900  of    5,360.    Elapsed: 0:20:15. Remaining: 1:40:20
  Batch   1,000  of    5,360.    Elapsed: 0:22:32. Remaining: 1:38:13
  Batch   1,100  of    5,360.    Elapsed: 0:24:48. Remaining: 1:36:02
  Batch   1,200  of    5,360.    Elapsed: 0:27:04. Remaining: 1:33:51
  Batch   1,300  of    5,360.    Elapsed: 0:29:20. Remaining: 1:31:37
  Batch   1,400  of    5,360.    

In [ ]:
import json

with open(os.path.join('/content/drive/MyDrive/dev-v1.1.json'), "r", encoding="utf-8") as reader:
    input_data = json.load(reader)["data"]


print_count = 0

print('Unpacking SQuAD Examples...')

print('Articles:')

examples = []

for entry in input_data:

    title = entry["title"]
    print('  ', title)

    for paragraph in entry["paragraphs"]:
        
        context_text = GoogleTranslator(source='en', target='fa').translate(paragraph["context"])
        
        for qa in paragraph["qas"]:
            
            ex = {}

            ex['qas_id'] = qa["id"]

            ex['question_text'] = GoogleTranslator(source='en', target='fa').translate(qa["question"])

            ex['answers'] = GoogleTranslator(source='en', target='fa').translate(qa["answers"])

            ex['title'] = title
            ex['context_text'] = GoogleTranslator(source='en', target='fa').translate(context_text)

            examples.append(ex)

print('DONE!')

Unpacking SQuAD Examples...
Articles:
   Super_Bowl_50
   Warsaw
   Normans
   Nikola_Tesla
   Computational_complexity_theory
   Teacher
   Martin_Luther
   Southern_California
   Sky_(United_Kingdom)
   Victoria_(Australia)
   Huguenot
   Steam_engine
   Oxygen
   1973_oil_crisis
   Apollo_program
   European_Union_law
   Amazon_rainforest
   Ctenophora
   Fresno,_California
   Packet_switching
   Black_Death
   Geology
   Newcastle_upon_Tyne
   Victoria_and_Albert_Museum
   American_Broadcasting_Company
   Genghis_Khan
   Pharmacy
   Immune_system
   Civil_disobedience
   Construction
   Private_school
   Harvard_University
   Jacksonville,_Florida
   Economic_inequality
   Doctor_Who
   University_of_Chicago
   Yuan_dynasty
   Kenya
   Intergovernmental_Panel_on_Climate_Change
   Chloroplast
   Prime_number
   Rhine
   Scottish_Parliament
   Islamism
   Imperialism
   United_Methodist_Church
   French_and_Indian_War
   Force
DONE!


In [ ]:
def good_update_interval(total_iters, num_desired_updates):

    exact_interval = total_iters / num_desired_updates

    order_of_mag = len(str(total_iters)) - 1

    round_mag = order_of_mag - 1

    update_interval = int(round(exact_interval, -round_mag))

    if update_interval == 0:
        update_interval = 1

    return update_interval

In [ ]:
import time
import torch

import logging

logging.getLogger("transformers.tokenization_utils_base").setLevel(logging.ERROR)

t0 = time.time()

start_positions = []
end_positions = []

num_clipped_answers = 0
num_impossible = 0

update_interval = good_update_interval(
            total_iters = len(examples), 
            num_desired_updates = 15
        )

print('Processing {:,} examples...'.format(len(examples)))

for (ex_num, ex) in enumerate(examples):

    if (ex_num % update_interval) == 0 and not (ex_num == 0):

        elapsed = format_time(time.time() - t0)
        
        ex_per_sec = (time.time() - t0) / ex_num
        remaining_sec = ex_per_sec * (len(examples) - ex_num)
        remaining = format_time(remaining_sec)

        print('  Example {:>7,}  of  {:>7,}.    Elapsed: {:}. Remaining: {:}'.format(ex_num, len(examples), elapsed, remaining))

    start_options = []
    end_options = []

    encoded_stored = False

    for answer in ex['answers']:

        answer_tokens = tokenizer.tokenize(answer['text'])

        sentinel_str = ' '.join(['[MASK]']*len(answer_tokens))

        start_char_i = answer['answer_start']
        end_char_i = start_char_i + len(answer['text'])

        context_w_sentinel = ex['context_text'][:start_char_i] + \
                            sentinel_str + \
                            ex['context_text'][end_char_i:]

        input_ids = tokenizer.encode(
            ex['question_text'], 
            context_w_sentinel,
            add_special_tokens = True, 
            pad_to_max_length = False,
            truncation = False,
        )

        mask_token_indeces = np.where(np.array(input_ids) == tokenizer.mask_token_id)[0]

        assert(len(mask_token_indeces) == len(answer_tokens))           

        start_index = mask_token_indeces[0]
        end_index = mask_token_indeces[-1]

        start_options.append(start_index)
        end_options.append(end_index)
    
    start_positions.append(start_options)
    end_positions.append(end_options)

print('DONE.  Tokenization took {:}'.format(format_time(time.time() - t0)))

Processing 10,570 examples...
  Example   1,000  of   10,570.    Elapsed: 0:00:07. Remaining: 0:01:04
  Example   2,000  of   10,570.    Elapsed: 0:00:13. Remaining: 0:00:55
  Example   3,000  of   10,570.    Elapsed: 0:00:19. Remaining: 0:00:49
  Example   4,000  of   10,570.    Elapsed: 0:00:29. Remaining: 0:00:47
  Example   5,000  of   10,570.    Elapsed: 0:00:39. Remaining: 0:00:44
  Example   6,000  of   10,570.    Elapsed: 0:00:48. Remaining: 0:00:36
  Example   7,000  of   10,570.    Elapsed: 0:00:57. Remaining: 0:00:29
  Example   8,000  of   10,570.    Elapsed: 0:01:05. Remaining: 0:00:21
  Example   9,000  of   10,570.    Elapsed: 0:01:12. Remaining: 0:00:13
  Example  10,000  of   10,570.    Elapsed: 0:01:21. Remaining: 0:00:05
DONE.  Tokenization took 0:01:27


In [ ]:
num_impossible = 0
num_clipped = 0

for (start_options, end_options) in zip(start_positions, end_positions):

    is_possible = False

    for i in range(0, len(start_options)):
        
        if (start_options[i] < 384) and (end_options[i] < 384):
            is_possible = True
        
        if (start_options[i] > 384) or (end_options[i] > 384):
            num_clipped += 1

    if not is_possible:
        num_impossible += 1

print('')

print('Samples w/ all answers clipped: {:,} of {:,} ({:.2%})'.format(num_impossible, len(examples), float(num_impossible) / float(len(examples))))

addtl_clipped = num_clipped - (num_impossible * 3)
total_answers = len(examples) * 3
print('\n    Additional clipped answers: {:,} of {:,}'.format(addtl_clipped, total_answers))

Samples w/ all answers clipped: 47 of 10,570 (0.44%)

    Additional clipped answers: 33 of 31,710


In [ ]:
import time
import torch

t0 = time.time()

all_input_ids = []
attention_masks = []
segment_ids = [] 

update_interval = good_update_interval(
            total_iters = len(examples), 
            num_desired_updates = 15
        )

print('Tokenizing {:,} examples...'.format(len(examples)))

for (ex_num, ex) in enumerate(examples):

    if (ex_num % update_interval) == 0 and not (ex_num == 0):

        elapsed = format_time(time.time() - t0)
        
        ex_per_sec = (time.time() - t0) / ex_num
        remaining_sec = ex_per_sec * (len(examples) - ex_num)
        remaining = format_time(remaining_sec)

        print('  Example {:>7,}  of  {:>7,}.    Elapsed: {:}. Remaining: {:}'.format(ex_num, len(examples), elapsed, remaining))

    encoded_dict = tokenizer.encode_plus(
        ex['question_text'], 
        ex['context_text'],
        add_special_tokens = True,
        max_length = 384, 
        pad_to_max_length = True,
        truncation = True,
        return_attention_mask = True,
        return_tensors = 'pt',
    )

    input_ids = encoded_dict['input_ids']
  
    all_input_ids.append(input_ids)

    attention_masks.append(encoded_dict['attention_mask'])    

    segment_ids.append(encoded_dict['token_type_ids'])

all_input_ids = torch.cat(all_input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
segment_ids = torch.cat(segment_ids, dim=0)

print('DONE.  Tokenization took {:}'.format(format_time(time.time() - t0)))

Tokenizing 10,570 examples...


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2132: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


  Example   1,000  of   10,570.    Elapsed: 0:00:02. Remaining: 0:00:22
  Example   2,000  of   10,570.    Elapsed: 0:00:04. Remaining: 0:00:19
  Example   3,000  of   10,570.    Elapsed: 0:00:07. Remaining: 0:00:17
  Example   4,000  of   10,570.    Elapsed: 0:00:09. Remaining: 0:00:15
  Example   5,000  of   10,570.    Elapsed: 0:00:12. Remaining: 0:00:14
  Example   6,000  of   10,570.    Elapsed: 0:00:15. Remaining: 0:00:12
  Example   7,000  of   10,570.    Elapsed: 0:00:18. Remaining: 0:00:09
  Example   8,000  of   10,570.    Elapsed: 0:00:21. Remaining: 0:00:07
  Example   9,000  of   10,570.    Elapsed: 0:00:23. Remaining: 0:00:04
  Example  10,000  of   10,570.    Elapsed: 0:00:26. Remaining: 0:00:01
DONE.  Tokenization took 0:00:27


In [ ]:
import time
import numpy as np

model.eval()

t0 = time.time()

pred_start = []
pred_end = []

num_test_samples = all_input_ids.shape[0]

batch_size = 16

num_batches = int(np.ceil(num_test_samples / batch_size))

print('Evaluating on {:,} test batches...'.format(num_batches))

batch_num = 0

for start_i in range(0, num_test_samples, batch_size):
    
    if ((batch_num % 50) == 0) and not (batch_num == 0):

        elapsed = format_time(time.time() - t0)
        
        batches_per_sec = (time.time() - t0) / batch_num
        remaining_sec = batches_per_sec * (num_batches - batch_num)
        remaining = format_time(remaining_sec)

        print('  Batch {:>7,}  of  {:>7,}.    Elapsed: {:}. Remaining: {:}'.format(batch_num, num_batches, elapsed, remaining))

    end_i = min(start_i + batch_size, num_test_samples)

    b_input_ids = all_input_ids[start_i:end_i, :]
    b_attn_masks = attention_masks[start_i:end_i, :]
    b_seg_ids = segment_ids[start_i:end_i, :]   

    b_input_ids = b_input_ids.to(device)
    b_attn_masks = b_attn_masks.to(device)
    b_seg_ids = b_seg_ids.to(device)
    
    with torch.no_grad():
        
        outputs = model(b_input_ids, 
                        attention_mask=b_attn_masks,
                        token_type_ids=b_seg_ids)
                        
    start_logits = outputs[0]
    end_logits = outputs[1]
    start_logits = start_logits.detach().cpu().numpy()
    end_logits = end_logits.detach().cpu().numpy()
    
    answer_start = np.argmax(start_logits, axis=1)
    answer_end = np.argmax(end_logits, axis=1)

    pred_start.append(answer_start)
    pred_end.append(answer_end)

    batch_num += 1

pred_start = np.concatenate(pred_start, axis=0)
pred_end = np.concatenate(pred_end, axis=0)

print('    DONE.')

print('\nEvaluation took {:.0f} seconds.'.format(time.time() - t0))

Evaluating on 661 test batches...
  Batch      50  of      661.    Elapsed: 0:00:20. Remaining: 0:04:09
  Batch     100  of      661.    Elapsed: 0:00:41. Remaining: 0:03:51
  Batch     150  of      661.    Elapsed: 0:01:03. Remaining: 0:03:33
  Batch     200  of      661.    Elapsed: 0:01:25. Remaining: 0:03:15
  Batch     250  of      661.    Elapsed: 0:01:47. Remaining: 0:02:56
  Batch     300  of      661.    Elapsed: 0:02:10. Remaining: 0:02:36
  Batch     350  of      661.    Elapsed: 0:02:32. Remaining: 0:02:15
  Batch     400  of      661.    Elapsed: 0:02:55. Remaining: 0:01:54
  Batch     450  of      661.    Elapsed: 0:03:19. Remaining: 0:01:33
  Batch     500  of      661.    Elapsed: 0:03:42. Remaining: 0:01:11
  Batch     550  of      661.    Elapsed: 0:04:05. Remaining: 0:00:50
  Batch     600  of      661.    Elapsed: 0:04:29. Remaining: 0:00:27
  Batch     650  of      661.    Elapsed: 0:04:53. Remaining: 0:00:05
    DONE.

Evaluation took 298 seconds.


In [ ]:
total_correct = 0

for i in range(0, len(pred_start)):

    match_options = []

    for j in range (0, len(start_positions[i])):
    
        matches = 0

        if pred_start[i] == start_positions[i][j]:
            matches += 1

        if pred_end[i] == end_positions[i][j]:
            matches += 1

        match_options.append(matches)

    total_correct += (max(match_options))


total_indeces = len(pred_start) + len(pred_end)

print('Correctly predicted indeces: {:,} of {:,} ({:.2%})'.format(
    total_correct,
    total_indeces,
    float(total_correct) / float(total_indeces)
))

Correctly predicted indeces: 16,378 of 21,140 (77.47%)


In [ ]:
f1s = []

for i in range(0, len(pred_start)):

    pred_span = set(range(pred_start[i], pred_end[i] + 1))


    f1_options = []

    for j in range (0, len(start_positions[i])):
    
        true_span = set(range(start_positions[i][j], end_positions[i][j] + 1))    

        num_same = len(pred_span.intersection(true_span))    

        if num_same == 0:
            f1_options.append(0)
            continue
    
        precision = float(num_same) / float(len(pred_span))
     
        recall = float(num_same) / float(len(true_span))

        f1 = (2 * precision * recall) / (precision + recall)

        f1_options.append(f1)

    f1s.append(max(f1_options))



print('Average F1 Score: {:.3f}'.format(np.mean(f1s)))

Average F1 Score: 0.791


In [ ]:
output_dir = './pretrained_model/'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

model_to_save = model.module if hasattr(model, 'module') else model  
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


Saving model to ./pretrained_model/


('./pretrained_model/tokenizer_config.json',
 './pretrained_model/special_tokens_map.json',
 './pretrained_model/vocab.txt',
 './pretrained_model/added_tokens.json')